<a href="https://colab.research.google.com/github/SouravKumarSinghnxtwave/Tutorial/blob/main/Sourav_Video_to_transcriptDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Load the code libraries

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install librosa

import whisper
import time
import librosa
import soundfile as sf
import re
import os

# model = whisper.load_model("tiny.en")
# model = whisper.load_model("base.en")
model = whisper.load_model("small.en") # load the small model
# model = whisper.load_model("medium.en")
# model = whisper.load_model("large")

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tyz5oof6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tyz5oof6
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=d0a2d20006ad0f998e1d8e68aac67d77ce3788104e1d31d8eed85cc9a785d383
  Stored in directory: /tmp/pip-ephem-wheel-cache-kjaqb1bv/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

100%|███████████████████████████████████████| 461M/461M [00:15<00:00, 32.1MiB/s]


##2. Give the application permission to mount the drive and create the folders

In [ ]:
# Create the Drive folders
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

# This will create the WhisperVideo files if they don't exist.
folders =  ["WhisperVideo/", "WhisperVideo/ProcessedVideo/", "WhisperVideo/TextFiles/", "WhisperVideo/AudioFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): # Create the folder if it does not exist
    os.mkdir(path)


Mounted at /content/drive


##3. Upload any video files you want transcribed in the "WhisperVideo" folder in your Google Drive.

##4. Extract audio from the video files and create a transcription

In [ ]:
# Load all the audio file paths in a Google Drive folder
from google.colab import drive
drive.mount("/content/drive", force_remount=True) # This will prompt for authorization.

# Get the list of video files from the WhisperVideo folder
video_files = os.listdir("/content/drive/MyDrive/WhisperVideo/")

# Loop through the video files and transcribe them
for video_file in video_files:

  # Skip the file if it is not a video format
  if not video_file.endswith((".mp4", ".mov", ".avi", ".mkv")):
    continue

  # Extract the audio from the video file using librosa
  video_path = "/content/drive/MyDrive/WhisperVideo/" + video_file
  audio_path = "/content/drive/MyDrive/WhisperVideo/AudioFiles/" + video_file[:-4] + ".wav" # Replace the video extension with .wav


  y, sr = librosa.load(video_path, sr=16000) # Load the audio with 16 kHz sampling rate
  sf.write(audio_path, y, sr) # Save the audio as a wav file

  # Transcribe the audio file using Whisper
  result = model.transcribe(audio_path)
  text = result["text"].strip()
  text = text.replace(". ", ".\n\n")

  # Save the transcription as a text file in Google Docs
  text_file = video_file[:-4] + ".txt" # Replace the video extension with .txt
  text_path = "/content/drive/MyDrive/WhisperVideo/TextFiles/" + text_file
  with open(text_path, "w") as f:
    f.write(text)

  # Move the video file to the ProcessedVideo folder
  processed_path = "/content/drive/MyDrive/WhisperVideo/ProcessedVideo/" + video_file
  os.rename(video_path, processed_path)

  # Print a message to indicate the progress
  print(f"Processed {video_file} and saved the transcription as {text_file}")

Mounted at /content/drive


<ipython-input-3-9f6f177c99e2>:20: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(video_path, sr=16000) # Load the audio with 16 kHz sampling rate
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processed video_sample.mp4 and saved the transcription as video_sample.txt
